In [1]:
import matplotlib as mpl
from matplotlib import pyplot as plt

import numpy as np

import sklearn

from sklearn.covariance import EmpiricalCovariance
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import OneClassSVM

import tensorflow as tf

In [2]:
import colorcet

conf_cmap = "cet_CET_L20"

In [3]:
def predict_rf_stdv(rf, X):
    Y = [e.predict(X) for e in rf.estimators_]
    Y = np.stack(Y, axis=1)
    
    return np.std(Y, axis=1)

In [4]:
def generate_toy_line(seed):
    rng = np.random.default_rng(seed)

    X1 = np.concatenate([np.linspace(0, 4, 5000), np.linspace(6, 10, 5000)])
    X2 = 1 + X1 * 0.5 + rng.normal(loc=0.0, scale=0.1, size=10000)
    X1 += rng.normal(loc=0.0, scale=0.1, size=10000)
    Y = X1*np.sqrt(1.25)
    
    return np.stack([X1, X2], axis=1), Y

def generate_toy_line_test():
    Xl1T, Xl2T = np.mgrid[-2:12:0.01, -1:8:0.01]
    Xl1T = Xl1T.flatten()
    Xl2T = Xl2T.flatten()

    return np.stack([Xl1T, Xl2T], axis=1)

def optimal_toy_line_reconstruction(Xl):
    return tf.constant([[0.0, 1.0]]) + tf.constant([[1.0, 0.5]]) * (
        tf.linalg.matmul((Xl - tf.constant([0.0, 1.0])), tf.constant([[1.0], [0.5]])) /
        tf.linalg.matmul(tf.constant([[1.0, 0.5]]), tf.constant([[1.0], [0.5]]))
    )

Xl, Yl = generate_toy_line(42)
XlV, YlV = generate_toy_line(24)
XlT = generate_toy_line_test()

In [5]:
def generate_toy_circle(seed):
    rng = np.random.default_rng(seed)
    
    Y = np.sin(np.linspace(0, np.pi * 8.0, 10000))
    
    X1 = np.sin(np.linspace(0, np.pi * 2.0, 10000)) * (5 + Y)
    X2 = np.cos(np.linspace(0, np.pi * 2.0, 10000)) * (5 + Y)
    
    X1 += rng.normal(loc=0.0, scale=0.1, size=10000)
    X2 += rng.normal(loc=0.0, scale=0.1, size=10000)
    
    return np.stack([X1, X2], axis=1), Y

def generate_toy_circle_test():
    Xl1T, Xl2T = np.mgrid[-7.5:7.5:0.01, -7.5:7.5:0.01]
    Xl1T = Xl1T.flatten()
    Xl2T = Xl2T.flatten()

    return np.stack([Xl1T, Xl2T], axis=1)

def optimal_toy_circle_reconstruction(Xc):
    angle = tf.math.atan2(Xc[:,0], Xc[:,1])
    
    Y = tf.math.sin(angle * 4.0)
    
    X1 = tf.math.sin(angle) * (5.0 + Y)
    X2 = tf.math.cos(angle) * (5.0 + Y)
    
    return tf.stack([X1, X2], axis=1)

Xc, Yc = generate_toy_circle(42)
XcV, YcV = generate_toy_circle(24)
XcT = generate_toy_circle_test()

In [6]:
def generate_toy_haystack(seed):
    rng = np.random.default_rng(seed)
    
    n = 10
    a = 2
    
    # https://stats.stackexchange.com/a/124554
    A = np.matrix([rng.normal(size=n) + rng.normal(size=1)*a for i in range(n)])
    A = A*np.transpose(A)
    D_half = np.diag(np.diag(A)**(-0.5))
    C = D_half*A*D_half
    covs = C
            
    Cn = rng.choice([-2, -1, 0, 0, 1, 2], size=10)
    
    Xn = rng.multivariate_normal(np.zeros(shape=10), covs, size=10000)
    
    Xn[:,3] = -0.42
    
    Yn = np.dot(Xn, Cn)
    
    return Xn, Yn

def generate_toy_haystack_validation(seed):
    rng = np.random.default_rng(seed)
    
    n = 10
    a = 2
    
    # https://stats.stackexchange.com/a/124554
    A = np.matrix([rng.normal(size=n) + rng.normal(size=1)*a for i in range(n)])
    A = A*np.transpose(A)
    D_half = np.diag(np.diag(A)**(-0.5))
    C = D_half*A*D_half
    covs = C
            
    Cn = rng.choice([-2, -1, 0, 0, 1, 2], size=10)
    
    _ = rng.integers(0, 2**30)
    rng = np.random.default_rng(rng.integers(0, 2**30))
    
    Xn = rng.multivariate_normal(np.zeros(shape=10), covs, size=10000)
    
    Xn[:,3] = -0.42
    
    Yn = np.dot(Xn, Cn)
    
    return Xn, Yn

def generate_toy_haystack_test(seed):
    rng = np.random.default_rng(seed)
    
    n = 10
    a = 2
    
    # https://stats.stackexchange.com/a/124554
    A = np.matrix([rng.normal(size=n) + rng.normal(size=1)*a for i in range(n)])
    A = A*np.transpose(A)
    D_half = np.diag(np.diag(A)**(-0.5))
    C = D_half*A*D_half
    covs = C
            
    Cn = rng.choice([-2, -1, 0, 0, 1, 2], size=10)
    
    rng = np.random.default_rng(rng.integers(0, 2**30))
    
    XnT = rng.multivariate_normal(np.zeros(shape=10), covs, size=10000)
    
    XnT[:,3] = (rng.random(size=10000) - 0.5) - 0.42
    # Provide some definitely ID points
    XnT[np.random.choice(len(XnT), size=len(XnT)//100),3] = -0.42
    
    return XnT

def optimal_toy_haystack_reconstruction(Xh):
    return (
        Xh * tf.constant([[1,1,1,0,1,1,1,1,1,1]], dtype=tf.float32) -
        tf.constant([[0,0,0,0.42,0,0,0,0,0,0]], dtype=tf.float32)
    )

Xh, Yh = generate_toy_haystack(42)
XhV, YhV = generate_toy_haystack_validation(42)
XhT = generate_toy_haystack_test(42)

In [7]:
def plot_line_toy_confidence(Xl, XlT, ClT, short, title):
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))

    ax.imshow(
        ClT.reshape(np.mgrid[-2:12:0.01, -1:8:0.01][0].shape).T, cmap=conf_cmap, vmin=0.0, vmax=1.2,
        extent=[-2,12,-1,8], origin="lower", interpolation="bicubic", rasterized=True,
    )

    ax.set_xlim(-2, 12)
    ax.set_ylim(-1, 8)
    
    ax.scatter(Xl[::250,0], Xl[::250,1], c='white', marker='x', lw=3, s=48)
    ax.scatter(Xl[::250,0], Xl[::250,1], c='black', marker='x')
    
    ax.text(
        0.5, 0.95, title, ha="center", va="top",
        size=20, c="white", bbox=dict(facecolor='black', alpha=0.25, edgecolor='white'),
        transform=ax.transAxes,
    )

    ax.axis('off')

    plt.savefig(f"confidence-line-{short}.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)

In [8]:
def plot_circle_toy_confidence(Xl, XlT, ClT, short):
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))

    ax.imshow(
        ClT.reshape(np.mgrid[-7.5:7.5:0.01, -7.5:7.5:0.01][0].shape).T, cmap=conf_cmap, vmin=0.0, vmax=1.2,
        extent=[-7.5,7.5,-7.5,7.5], origin="lower", interpolation="bicubic", rasterized=True,
    )

    ax.set_xlim(-7.5, 7.5)
    ax.set_ylim(-7.5, 7.5)
    
    ax.scatter(Xl[::250,0], Xl[::250,1], c='white', marker='x', lw=3, s=48)
    ax.scatter(Xl[::250,0], Xl[::250,1], c='black', marker='x')

    ax.axis('off')

    plt.savefig(f"confidence-circle-{short}.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)

In [9]:
def plot_haystack_toy_confidence(XlT, ClT, short):
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    ax.set_xticks([-0.42-0.4, -0.42, -0.42+0.4])
    ax.set_xticklabels([r"$-0.4 \sigma$", "const", "$+0.4 \sigma$"])
    ax.get_yaxis().set_visible(False)
    
    ax.axvline(-0.42, c='white', lw=7, zorder=-1)
    
    ax.scatter(XlT[:, 3], ClT, c='white', s=6, rasterized=True)
    ax.scatter(XlT[:, 3], ClT, c='black', s=1, rasterized=True)
    
    ax.imshow(
        [[x/100, x/100] for x in range(100, -1, -1)], cmap=conf_cmap, interpolation='bicubic',
        vmin=0.0, vmax=1.2, extent=[-0.93, 0.09, -0.01, 1.01], zorder=-2,
    )
    
    class TruncatedColormap(mpl.colors.Colormap):
        def __init__(self, cmap):
            self.cmap = cmap
            self.N = cmap.N
        
        def __call__(self, X, alpha=None, bytes=False):
            return self.cmap.__call__(X/1.2, alpha=alpha, bytes=bytes)
    
    cb = fig.colorbar(mpl.cm.ScalarMappable(
        norm=None, cmap=TruncatedColormap(mpl.cm.get_cmap(conf_cmap)),
    ), ax=ax, ticks=[0.1, 0.9])
    cb.set_label("confidence level $c$", labelpad=-13)
    cb.outline.set_visible(False)
    
    plt.savefig(f"confidence-haystack-{short}.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)

In [10]:
PlV = np.sort(np.sum((optimal_toy_line_reconstruction(XlV) - XlV)**2, axis=1))

ClT = 1.0 - np.searchsorted(
    PlV,
    np.sum((optimal_toy_line_reconstruction(XlT) - XlT)**2, axis=1),
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "aa-optimal", "Ideal OOD Detection Reference")

""" """

PcV = np.sort(np.sum((optimal_toy_circle_reconstruction(XcV) - XcV)**2, axis=1))

CcT = 1.0 - np.searchsorted(
    PcV,
    np.sum((optimal_toy_circle_reconstruction(XcT) - XcT)**2, axis=1),
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "aa-optimal")

""" """

PhV = np.sort(np.sum((optimal_toy_haystack_reconstruction(XhV) - XhV)**2, axis=1))

ChT = 1.0 - np.searchsorted(
    PhV,
    np.sum((optimal_toy_haystack_reconstruction(XhT) - XhT)**2, axis=1),
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "aa-optimal")

2023-03-24 20:45:08.392034: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 20:45:08.393412: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
Xl_cov = EmpiricalCovariance().fit(Xl)
PlV = np.sort(Xl_cov.mahalanobis(XlV))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_cov.mahalanobis(XlT), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "mahalanobis", "Mahalanobis distance")

""" """

Xc_cov = EmpiricalCovariance().fit(Xc)
PcV = np.sort(Xc_cov.mahalanobis(XcV))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_cov.mahalanobis(XcT), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "mahalanobis")

""" """

Xh_cov = EmpiricalCovariance().fit(Xh)
PhV = np.sort(Xh_cov.mahalanobis(XhV))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_cov.mahalanobis(XhT), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "mahalanobis")

In [12]:
Xl_pca = PCA(n_components=0.95, random_state=42).fit(Xl)
PlV = np.sort(np.sum((Xl_pca.inverse_transform(Xl_pca.transform(XlV)) - XlV)**2, axis=1))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(np.sum((Xl_pca.inverse_transform(Xl_pca.transform(XlT)) - XlT)**2, axis=1), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "truncated-pca", "Truncated PCA")

""" """

Xc_pca = PCA(n_components=1, random_state=42).fit(Xc)
PcV = np.sort(np.sum((Xc_pca.inverse_transform(Xc_pca.transform(XcV)) - XcV)**2, axis=1))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(np.sum((Xc_pca.inverse_transform(Xc_pca.transform(XcT)) - XcT)**2, axis=1), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "truncated-pca")

""" """

Xh_pca = PCA(n_components=0.95, random_state=42).fit(Xh)
PhV = np.sort(np.sum((Xh_pca.inverse_transform(Xh_pca.transform(XhV)) - XhV)**2, axis=1))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(np.sum((Xh_pca.inverse_transform(Xh_pca.transform(XhT)) - XhT)**2, axis=1), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "truncated-pca")

In [13]:
Xl_svm = OneClassSVM(kernel="rbf").fit(Xl)
PlV = np.sort(Xl_svm.score_samples(XlV))

ClT = np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_svm.score_samples(XlT), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "svm-rbf", 'SVM(kernel="rbf")')

""" """

Xc_svm = OneClassSVM(kernel="rbf").fit(Xc)
PcV = np.sort(Xc_svm.score_samples(XcV))

CcT = np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_svm.score_samples(XcT), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "svm-rbf")

""" """

Xh_svm = OneClassSVM(kernel="rbf").fit(Xh)
PhV = np.sort(Xh_svm.score_samples(XhV))

ChT = np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_svm.score_samples(XhT), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "svm-rbf")

In [14]:
Xl_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xl, Xl)
PlV = np.sort(np.sum((Xl_mlp.predict(XlV) - XlV)**2, axis=1))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(np.sum((Xl_mlp.predict(XlT) - XlT)**2, axis=1), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "aa-relu", "Auto-Associative-ReLU")

""" """

Xc_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xc, Xc)
PcV = np.sort(np.sum((Xc_mlp.predict(XcV) - XcV)**2, axis=1))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(np.sum((Xc_mlp.predict(XcT) - XcT)**2, axis=1), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "aa-relu")

""" """

Xh_mlp = MLPRegressor(hidden_layer_sizes=[16, 8, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xh, Xh)
PhV = np.sort(np.sum((Xh_mlp.predict(XhV) - XhV)**2, axis=1))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(np.sum((Xh_mlp.predict(XhT) - XhT)**2, axis=1), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "aa-relu")

In [15]:
Xl_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xl, Xl)
Xl_cov = EmpiricalCovariance().fit((Xl_mlp.predict(Xl) - Xl))
PlV = np.sort(Xl_cov.mahalanobis((Xl_mlp.predict(XlV) - XlV)))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_cov.mahalanobis((Xl_mlp.predict(XlT) - XlT)), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "aa-relu-mahalanobis", "Mahalanobis(AA-ReLU)")

""" """

Xc_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xc, Xc)
Xc_cov = EmpiricalCovariance().fit((Xc_mlp.predict(Xc) - Xc))
PcV = np.sort(Xc_cov.mahalanobis((Xc_mlp.predict(XcV) - XcV)))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_cov.mahalanobis((Xc_mlp.predict(XcT) - XcT)), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "aa-relu-mahalanobis")

""" """

Xh_mlp = MLPRegressor(hidden_layer_sizes=[16, 8, 16], activation="relu", solver="adam", max_iter=5000, random_state=42).fit(Xh, Xh)
Xh_cov = EmpiricalCovariance().fit((Xh_mlp.predict(Xh) - Xh))
PhV = np.sort(Xh_cov.mahalanobis((Xh_mlp.predict(XhV) - XhV)))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_cov.mahalanobis((Xh_mlp.predict(XhT) - XhT)), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "aa-relu-mahalanobis")

In [16]:
Xl_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xl, Xl)
PlV = np.sort(np.sum((Xl_mlp.predict(XlV) - XlV)**2, axis=1))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(np.sum((Xl_mlp.predict(XlT) - XlT)**2, axis=1), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "aa-sigmoid", "Auto-Associative-Sigmoid")

""" """

Xc_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xc, Xc)
PcV = np.sort(np.sum((Xc_mlp.predict(XcV) - XcV)**2, axis=1))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(np.sum((Xc_mlp.predict(XcT) - XcT)**2, axis=1), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "aa-sigmoid")

""" """

Xh_mlp = MLPRegressor(hidden_layer_sizes=[16, 8, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xh, Xh)
PhV = np.sort(np.sum((Xh_mlp.predict(XhV) - XhV)**2, axis=1))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(np.sum((Xh_mlp.predict(XhT) - XhT)**2, axis=1), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "aa-sigmoid")

In [17]:
Xl_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xl, Xl)
Xl_cov = EmpiricalCovariance().fit((Xl_mlp.predict(Xl) - Xl))
PlV = np.sort(Xl_cov.mahalanobis((Xl_mlp.predict(XlV) - XlV)))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_cov.mahalanobis((Xl_mlp.predict(XlT) - XlT)), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "aa-sigmoid-mahalanobis", "Mahalanobis(AA-Sigmoid)")

""" """

Xc_mlp = MLPRegressor(hidden_layer_sizes=[16, 1, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xc, Xc)
Xc_cov = EmpiricalCovariance().fit((Xc_mlp.predict(Xc) - Xc))
PcV = np.sort(Xc_cov.mahalanobis((Xc_mlp.predict(XcV) - XcV)))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_cov.mahalanobis((Xc_mlp.predict(XcT) - XcT)), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "aa-sigmoid-mahalanobis")

""" """

Xh_mlp = MLPRegressor(hidden_layer_sizes=[16, 8, 16], activation="logistic", solver="adam", max_iter=5000, random_state=42).fit(Xh, Xh)
Xh_cov = EmpiricalCovariance().fit((Xh_mlp.predict(Xh) - Xh))
PhV = np.sort(Xh_cov.mahalanobis((Xh_mlp.predict(XhV) - XhV)))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_cov.mahalanobis((Xh_mlp.predict(XhT) - XhT)), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "aa-sigmoid-mahalanobis")

In [18]:
Xl_lof = LocalOutlierFactor(novelty=True).fit(Xl)
PlV = np.sort(Xl_lof.score_samples(XlV))

ClT = np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_lof.score_samples(XlT), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "lof", "Local Outlier Factor")

""" """

Xc_lof = LocalOutlierFactor(novelty=True).fit(Xc)
PcV = np.sort(Xc_lof.score_samples(XcV))

CcT = np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_lof.score_samples(XcT), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "lof")

""" """

Xh_lof = LocalOutlierFactor(novelty=True).fit(Xh)
PhV = np.sort(Xh_lof.score_samples(XhV))

ChT = np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_lof.score_samples(XhT), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "lof")

In [19]:
Xl_iso = IsolationForest(random_state=42).fit(Xl)
PlV = np.sort(Xl_iso.score_samples(XlV))

ClT = np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_iso.score_samples(XlT), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "isolation", "Isolation Forest")

""" """

Xc_iso = IsolationForest(random_state=42).fit(Xc)
PcV = np.sort(Xc_iso.score_samples(XcV))

CcT = np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_iso.score_samples(XcT), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "isolation")

""" """

Xh_iso = IsolationForest(random_state=42).fit(Xh)
PhV = np.sort(Xh_iso.score_samples(XhV))

ChT = np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_iso.score_samples(XhT), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "isolation")

In [20]:
Xl_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xl[::100], Yl[::100])
PlV = np.sort(Xl_gp.predict(XlV, return_std=True)[1])

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_gp.predict(XlT, return_std=True)[1], 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "gp-coarse-xy", r"GP($X \rightarrow Y$) @ 1%")

""" """

Xc_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xc[::100], Yc[::100])
PcV = np.sort(Xc_gp.predict(XcV, return_std=True)[1])

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_gp.predict(XcT, return_std=True)[1], 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "gp-coarse-xy")

""" """

Xh_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xh[::100], Yh[::100])
PhV = np.sort(Xh_gp.predict(XhV, return_std=True)[1])

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_gp.predict(XhT, return_std=True)[1], 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "gp-coarse-xy")

/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [21]:
Xl_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xl[::100], Xl[::100])
PlV = np.sort(np.sum(Xl_gp.predict(XlV, return_std=True)[1], axis=1))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(np.sum(Xl_gp.predict(XlT, return_std=True)[1], axis=1), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "gp-coarse-xx", r"AA-GP$_{uq}$($X$) @ 1%")

""" """

Xc_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xc[::100], Xc[::100])
PcV = np.sort(np.sum(Xc_gp.predict(XcV, return_std=True)[1], axis=1))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(np.sum(Xc_gp.predict(XcT, return_std=True)[1], axis=1), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "gp-coarse-xx")

""" """

Xh_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xh[::10], Xh[::10])
PhV = np.sort(np.sum(Xh_gp.predict(XhV, return_std=True)[1], axis=1))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(np.sum(Xh_gp.predict(XhT, return_std=True)[1], axis=1), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "gp-coarse-xx")

/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [22]:
Xl_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xl[::10], Xl[::10])
Xl_cov = EmpiricalCovariance().fit(Xl_gp.predict(Xl) - Xl)
PlV = np.sort(Xl_cov.mahalanobis(Xl_gp.predict(XlV) - XlV))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_cov.mahalanobis(Xl_gp.predict(XlT) - XlT), 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "gp-xx-mahalanobis", r"Mahalanobis(AA-GP$_{err}$($X$))@10%")

""" """

Xc_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xc[::10], Xc[::10])
Xc_cov = EmpiricalCovariance().fit(Xc_gp.predict(Xc) - Xc)
PcV = np.sort(Xc_cov.mahalanobis(Xc_gp.predict(XcV) - XcV))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_cov.mahalanobis(Xc_gp.predict(XcT) - XcT), 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "gp-xx-mahalanobis")

""" """

Xh_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xh[::10], Xh[::10])
Xh_cov = EmpiricalCovariance().fit(Xh_gp.predict(Xh) - Xh)
PhV = np.sort(Xh_cov.mahalanobis(Xh_gp.predict(XhV) - XhV))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_cov.mahalanobis(Xh_gp.predict(XhT) - XhT), 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "gp-xx-mahalanobis")

/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [23]:
Xl_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xl[::10], Yl[::10])
PlV = np.sort(Xl_gp.predict(XlV, return_std=True)[1])

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_gp.predict(XlT, return_std=True)[1], 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "gp", r"GP($X \rightarrow Y$) @ 10%")

""" """

Xc_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xc[::10], Yc[::10])
PcV = np.sort(Xc_gp.predict(XcV, return_std=True)[1])

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_gp.predict(XcT, return_std=True)[1], 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "gp")

""" """

Xh_gp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xh[::10], Yh[::10])
PhV = np.sort(Xh_gp.predict(XhV, return_std=True)[1])

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_gp.predict(XhT, return_std=True)[1], 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "gp")

/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspace/msc-tcm/miniconda3/envs/demo/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [24]:
rng = np.random.default_rng(24)

Xl2 = np.concatenate([Xl, rng.normal(loc=Xl, scale=2.0)])
Yl2 = np.concatenate([Yl, rng.normal(loc=Yl, scale=4.0)])

Xc2 = np.concatenate([Xc, rng.normal(loc=Xc, scale=7.5)])
Yc2 = np.concatenate([Yc, rng.normal(loc=Yc, scale=2.0)])

Xh2 = np.concatenate([Xh, rng.normal(loc=Xh, scale=4.0)])
Yh2 = np.concatenate([Yh, rng.normal(loc=Yh, scale=4.0)])

In [25]:
Xl_ncp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xl2[::25], Yl2[::25])
PlV = np.sort(Xl_ncp.predict(XlV, return_std=True)[1])

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 5),
    np.round(Xl_ncp.predict(XlT, return_std=True)[1], 5)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "nc-gp", "Noise-Contrastive GP 10%")

""" """

Xc_ncp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xc2[::25], Yc2[::25])
PcV = np.sort(Xc_ncp.predict(XcV, return_std=True)[1])

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 5),
    np.round(Xc_ncp.predict(XcT, return_std=True)[1], 5)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "nc-gp")

""" """

Xh_ncp = GaussianProcessRegressor(kernel=RBF()+WhiteKernel()).fit(Xh2[::25], Yh2[::25])
PhV = np.sort(Xh_ncp.predict(XhV, return_std=True)[1])

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 5),
    np.round(Xh_ncp.predict(XhT, return_std=True)[1], 5)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "nc-gp")

In [26]:
Xl_rf = RandomForestRegressor(n_estimators=10, max_depth=int(np.sqrt(len(Xl))), random_state=42).fit(Xl, Yl)
PlV = np.sort(predict_rf_stdv(Xl_rf, XlV))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 3),
    np.round(predict_rf_stdv(Xl_rf, XlT), 3)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "rf", "Random Forest")

""" """

Xc_rf = RandomForestRegressor(n_estimators=10, max_depth=int(np.sqrt(len(Xc))), random_state=42).fit(Xc, Yc)
PcV = np.sort(predict_rf_stdv(Xc_rf, XcV))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 3),
    np.round(predict_rf_stdv(Xc_rf, XcT), 3)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "rf")

""" """

Xh_rf = RandomForestRegressor(n_estimators=10, max_depth=int(np.sqrt(len(Xh))), random_state=42).fit(Xh, Yh)
PhV = np.sort(predict_rf_stdv(Xh_rf, XhV))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 3),
    np.round(predict_rf_stdv(Xh_rf, XhT), 3)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "rf")

In [27]:
Xl_rf = RandomForestRegressor(n_estimators=100, max_depth=int(np.sqrt(len(Xl2))), random_state=42).fit(Xl2, Yl2)
PlV = np.sort(predict_rf_stdv(Xl_rf, XlV))

ClT = 1.0 - np.searchsorted(
    np.round(PlV, 3),
    np.round(predict_rf_stdv(Xl_rf, XlT), 3)
) / len(PlV)

plot_line_toy_confidence(Xl, XlT, ClT, "nc-rf", "Noise-Contrastive RF")

""" """

Xc_rf = RandomForestRegressor(n_estimators=100, max_depth=int(np.sqrt(len(Xc2))), random_state=42).fit(Xc2, Yc2)
PcV = np.sort(predict_rf_stdv(Xc_rf, XcV))

CcT = 1.0 - np.searchsorted(
    np.round(PcV, 3),
    np.round(predict_rf_stdv(Xc_rf, XcT), 3)
) / len(PcV)

plot_circle_toy_confidence(Xc, XcT, CcT, "nc-rf")

""" """

Xh_rf = RandomForestRegressor(n_estimators=100, max_depth=int(np.sqrt(len(Xh2))), random_state=42).fit(Xh2, Yh2)
PhV = np.sort(predict_rf_stdv(Xh_rf, XhV))

ChT = 1.0 - np.searchsorted(
    np.round(PhV, 3),
    np.round(predict_rf_stdv(Xh_rf, XhT), 3)
) / len(PhV)

plot_haystack_toy_confidence(XhT, ChT, "nc-rf")